Most examples work across multiple plotting backends, this example is also available for:

* [Matplotlib NYC Taxi Connection](../matplotlib/nyc_taxi_connections.ipynb)

In [ ]:
import numpy as np
import networkx as nx
import holoviews as hv
from holoviews import opts
from holoviews import dim

from holoviews.element.graphs import layout_nodes
from bokeh.sampledata.airport_routes import routes, airports

hv.extension('bokeh')

# Declare data

In [ ]:
# Create dataset indexed by AirportID and with additional value dimension
airports = hv.Dataset(airports, ['AirportID'], ['Name', 'IATA', 'City'])
source_airports = list(airports.select(City='New York').data.AirportID)

# Add connections count to routes then aggregate and select just routes connecting to NYC
routes['connections'] = 1
nyc_graph = hv.Graph((routes, airports), ['SourceID', "DestinationID"], ['connections'], label='NYC Airport Connections').aggregate(function=np.count_nonzero).select(SourceID=source_airports)

# Lay out graph weighting and weight by the number of connections
np.random.seed(14)
graph = layout_nodes(nyc_graph, layout=nx.layout.fruchterman_reingold_layout, kwargs={'weight': 'connections'})
labels = hv.Labels(graph.nodes, ['x', 'y'], ['IATA', 'City'])
nyc_labels = labels.select(City='New York').options(text_color='white', yoffset=0.05, text_font_size='16pt')
other_labels = labels[labels['City']!='New York'].options(text_color='white', text_font_size='8pt')

## Plot

In [ ]:
plot = (graph * nyc_labels * other_labels).relabel('NYC Airport Connections')

plot.options(
    opts.Graph(bgcolor='gray', edge_color=dim('SourceID').categorize({3697: 'red', 3797: 'blue'}, 'gray'), 
               node_color=dim('index').categorize({3697: 'red', 3797: 'blue'}, 'gray'), width=800, height=800, 
               xaxis=None, yaxis=None, xlim=(-1.2, 1.2), ylim=(-1.2, 1.2))
)